## Python set-up

In [1]:
# Lib imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
from datetime import datetime

# Local imports
from abs_data_capture import (
    AbsLandingPage,
    get_abs_data,
    create_selector_series_dataframe,
    create_all_series_dataframe,
    clear_cache,
    metacol
)

# Clearing cache data from 
clear_cache()

## Get data from ABS

In [3]:
LANDING_PAGE_LABOUR_FORCE = AbsLandingPage(
    theme="labour",
    parent_topic="employment-and-unemployment",
    topic="labour-force-australia",
)

abs_dict_lf = get_abs_data(LANDING_PAGE_LABOUR_FORCE)

LANDING_PAGE_JOB_VACANCIES = AbsLandingPage(
    theme="labour",
    parent_topic="employment-and-unemployment",
    topic="job-vacancies-australia",
)

abs_dict_jv = get_abs_data(LANDING_PAGE_JOB_VACANCIES)

## Getting Dataframe with ABS Info

In [10]:
labour_trend = {
        "1": metacol.table,
        "Persons": metacol.did,
        "Trend":metacol.stype
    }

labour_sa = {
        "1": metacol.table,
        "Persons": metacol.did,
        "Seasonally Adjusted":metacol.stype
    }

job_vacancy_sa = {
        "1": metacol.table,
        "Seasonally Adjusted": metacol.stype,
    }

underemployment_trend = {
        "22": metacol.table,
        "A85256589V": metacol.id,
    }

underemployment_sa = {
        "22": metacol.table,
        "A85255725J": metacol.id,
    }

In [11]:
df_labour_trend = create_selector_series_dataframe(abs_dict_lf, labour_trend)
df_labour_sa = create_selector_series_dataframe(abs_dict_lf, labour_sa)
df_job_vacancy_sa = create_selector_series_dataframe(abs_dict_jv, job_vacancy_sa)
df_under_trend = create_selector_series_dataframe(abs_dict_lf,underemployment_trend)
df_under_sa = create_selector_series_dataframe(abs_dict_lf,underemployment_sa)
df_labour_trend.index = df_labour_trend.index.to_timestamp()
df_labour_sa.index = df_labour_sa.index.to_timestamp()
df_job_vacancy_sa.index = df_job_vacancy_sa.index.to_timestamp()
df_under_trend.index = df_under_trend.index.to_timestamp()
df_under_sa.index = df_under_sa.index.to_timestamp()


In [14]:
def annualise_rates(data: pd.Series, periods: int|float = 12) -> pd.Series:
    """Annualise a growth rate for a period."""
    return (((1 + data.pct_change(periods=1)) ** periods) - 1) * 100

def annualise_percentages(data: pd.Series, periods: int|float = 12) -> pd.Series:
    """Annualise a growth rate (expressed as a percentage) for a period."""
    rates = data / 100.0
    return annualise_rates(rates, periods)

def absolute_change(data: pd.Series, periods: int|float = 1) -> pd.Series:
    """Calculate an n-periods absolute change."""
    return data - data.shift(periods)

def percent_change(data: pd.Series, periods: int|float = 1) -> pd.Series:
    """Calculate an n-periods percentage change."""
    return (data / data.shift(periods) - 1) * 100

def create_index_series(series: pd.Series, base_date: str) -> pd.Series:
    """ Return a series with values adjusted by the index."""
    
    # Check if base_date is in the series index
    if base_date not in series.index:
        raise ValueError("base_date not found in the series index")

    # Calculate the index factor
    base_value = series.loc[base_date]
    if base_value == 0:
        raise ValueError("base_value at base_date is zero, cannot create index")

    indexed_series = (series / base_value) * 100
    return indexed_series


## Getting Plots for PDF

In [15]:
# Use a relative path for the PDF
pdf_path = 'Labour_Market_Report.pdf'
pdf = PdfPages(pdf_path)

In [40]:
# Figure 1: Labour Market Summary
# Creating the plot with specified dimensions
fig, ax1 = plt.subplots(figsize=(14, 8))

# Adding a page title above the plot
fig.suptitle('Figure 1: Labour Market, Summary', fontsize=16, fontweight='bold', y=0.95)

# Plotting unemployment rate with a red line
ax1.plot(df_labour_sa.index, df_labour_sa['Unemployment rate'], label='Unemployment Rate', color='tab:red')

# Plotting underemployment rate with a blue line
ax1.plot(df_under_sa.index, df_under_sa['Underemployment rate (proportion of labour force)'], label='Underemployment Rate (Proportion of Labour Force)', color='tab:blue')

# Creating a secondary y-axis for the participation rate
ax2 = ax1.twinx()
ax2.plot(df_labour_sa.index, df_labour_sa['Participation rate'], label='Participation Rate (RHS)', color='tab:green', linestyle='--')

# Setting x-axis and y-axis labels
ax1.set_xlabel('Date')
ax1.set_ylabel('(%)')
ax2.set_ylabel('(%)', color='tab:green')

# Setting a title for the axes
ax1.set_title('Australia Labour Market')

# Formatting the x-axis to show month/year and adjusting interval to display a tick every six months
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

# Setting x-axis limits to start from a specific date
ax1.set_xlim([datetime(2009, 10, 1), df_labour_sa.index.max()])

# Adding grid lines to the primary axis
ax1.grid(True)

# Rotating x-axis labels for better readability
plt.setp(ax1.get_xticklabels(), rotation=90)

# Adding legends for both primary and secondary y-axes
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Adjusting layout to ensure the plot is centered and fits well within an A4 page
plt.tight_layout(pad=2.0)

# Saving the current figure into the PDF
pdf.savefig(fig, bbox_inches='tight')

# Close the plot
plt.close()

In [41]:
# Figure 2: Labour Force Summary
fig, ax1 = plt.subplots(figsize=(14, 8))

# Add page title
fig.suptitle('Figure 2: Labour Force, Summary', fontsize=16, fontweight='bold', y=0.95)

# Plot total labour force and total employed
ax1.plot(df_labour_sa.index, df_labour_sa['Labour force total'], label='Labour Force Total', color='tab:blue')
ax1.plot(df_labour_sa.index, df_labour_sa['Employed total'], label='Employed Total', color='tab:green')

# Create a second y-axis for total unemployed
ax2 = ax1.twinx()
ax2.plot(df_labour_sa.index, df_labour_sa['Unemployed total'], label='Unemployed Total (RHS)', color='tab:red', linestyle='--')

# Set labels for x and y axes
ax1.set_xlabel('Date')
ax1.set_ylabel('(Thousands)')
ax2.set_ylabel('(Thousands)', color='tab:red')

# Set title for the axes
ax1.set_title('Labour Force')

# Format x-axis to show month/year
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

# Set x-axis limits
ax1.set_xlim([datetime(2009, 10, 1), df_labour_sa.index.max()])

# Add grid to the main axis
ax1.grid(True)

# Rotate x-axis labels
plt.setp(ax1.get_xticklabels(), rotation=90)

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Adjust layout to ensure the plot is centered and well-fitted on the A4 page
plt.tight_layout(pad=2.0)

# Save the current figure to the PDF
pdf.savefig(fig, bbox_inches='tight')

# Close the plot
plt.close()


In [42]:
# Figure 3: Employment Growth
fig, ax1 = plt.subplots(figsize=(14, 8))

# Calculating series for plotting
total_employee_growth = percent_change(df_labour_sa['Employed total'], 12)
full_time_employee_growth = percent_change(df_labour_sa['> Employed full-time'], 12)
part_time_employee_growth = percent_change(df_labour_sa['> Employed part-time'], 12)

# Add page title
fig.suptitle('Figure 3: Employment Growth', fontsize=16, fontweight='bold', y=0.95)

# Plot lines
ax1.plot(total_employee_growth.index, total_employee_growth, label='Total', color='tab:red')
ax1.plot(full_time_employee_growth.index, full_time_employee_growth, label='Full-Time', color='tab:blue')
ax1.plot(part_time_employee_growth.index, part_time_employee_growth, label='Part-Time', color='tab:green')

# Set labels for x and y axes
ax1.set_xlabel('Date')
ax1.set_ylabel('(%)')

# Set title for the axes
ax1.set_title('Employment Growth (YoY)')

# Format x-axis to show month/year
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

# Set x-axis limits
ax1.set_xlim([datetime(2009, 10, 1), total_employee_growth.index.max()])

# Add grid to the main axis
ax1.grid(True)

# Rotate x-axis labels
plt.setp(ax1.get_xticklabels(), rotation=90)

# Add legends
ax1.legend(loc='upper left')

# Adjust layout to ensure the plot is centered and well-fitted on the A4 page
plt.tight_layout(pad=2.0)

# Save the current figure to the PDF
pdf.savefig(fig, bbox_inches='tight')

# Close the plot
plt.close()

In [57]:
# Figure 4: Employment Change
fig, ax1 = plt.subplots(figsize=(14, 8))

# Calculating series for plotting
abs_total_employee_growth = absolute_change(df_labour_sa['Employed total'], 1)
abs_full_time_employee_growth = absolute_change(df_labour_sa['> Employed full-time'], 1)
abs_part_time_employee_growth = absolute_change(df_labour_sa['> Employed part-time'], 1)

# Add page title
fig.suptitle('Figure 4: Employment Change', fontsize=16, fontweight='bold', y=0.95)

# Plot lines
ax1.plot(abs_total_employee_growth.index, abs_total_employee_growth, label='Total', color='tab:red')
ax1.bar(abs_full_time_employee_growth.index, abs_full_time_employee_growth, 25, label='Full-Time', color='tab:blue')
ax1.bar(abs_part_time_employee_growth.index, abs_part_time_employee_growth, 25, label='Part-Time', color='tab:green')

# Set labels for x and y axes
ax1.set_xlabel('Date')
ax1.set_ylabel('Changes')

# Set title for the axes
ax1.set_title('Employment Changes (MoM)')

# Format x-axis to show month/year
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

# Set x-axis limits
ax1.set_xlim([datetime(2009, 10, 1), abs_total_employee_growth.index.max()])

# Add grid to the main axis
ax1.grid(True)

# Rotate x-axis labels
plt.setp(ax1.get_xticklabels(), rotation=90)

# Add legends
ax1.legend(loc='upper left')

# Adjust layout to ensure the plot is centered and well-fitted on the A4 page
plt.tight_layout(pad=2.0)

# Save the current figure to the PDF
pdf.savefig(fig, bbox_inches='tight')

# Close the plot
plt.close()

In [16]:
# Figure 5: Employment Comparision
fig, ax1 = plt.subplots(figsize=(14, 8))

# Calculating series for plotting
idx_total_employee = create_index_series(df_labour_sa['Employed total'], '2000-01-01')
idx_full_time_employee = create_index_series(df_labour_sa['> Employed full-time'], '2000-01-01')
idx_part_time_employee = create_index_series(df_labour_sa['> Employed part-time'], '2000-01-01')

# Add page title
fig.suptitle('Figure 5: Employment Comparision', fontsize=16, fontweight='bold', y=0.95)

# Plot lines
ax1.plot(idx_total_employee.index, idx_total_employee, label='Total', color='tab:red')
ax1.plot(idx_full_time_employee.index, idx_full_time_employee, label='Full-Time', color='tab:blue')
ax1.plot(idx_part_time_employee.index, idx_part_time_employee, label='Part-Time', color='tab:green')

# Set labels for x and y axes
ax1.set_xlabel('Date')
ax1.set_ylabel('Index')

# Set title for the axes
ax1.set_title('Employment Comparision: Level − Jan/00 = 100')

# Format x-axis to show month/year
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

# Set x-axis limits
ax1.set_xlim([datetime(2009, 10, 1), idx_total_employee.index.max()])

# Calculate the maximum value among the series
max_value = np.max([idx_total_employee.max(), idx_full_time_employee.max(), idx_part_time_employee.max()])

# Set y-axis limits with minimum value as 100 and maximum value as the maximum value among the series
ax1.set_ylim([100, max_value + 10])

# Add grid to the main axis
ax1.grid(True)

# Rotate x-axis labels
plt.setp(ax1.get_xticklabels(), rotation=90)

# Add legends
ax1.legend(loc='upper left')

# Adjust layout to ensure the plot is centered and well-fitted on the A4 page
plt.tight_layout(pad=2.0)

# Save the current figure to the PDF
pdf.savefig(fig, bbox_inches='tight')

# Close the plot
plt.close()

In [21]:
# Figure 6: Job Vacancies
fig, ax1 = plt.subplots(figsize=(14, 8))

# Add page title
fig.suptitle('Figure 6: Job Vacancies', fontsize=16, fontweight='bold', y=0.95)

# Plot lines
ax1.plot(df_job_vacancy_sa.index, df_job_vacancy_sa['Job Vacancies'], label='Job Vacancies', color='tab:red')

# Set labels for x and y axes
ax1.set_xlabel('Date')
ax1.set_ylabel('Thousands')

# Set title for the axes
ax1.set_title('Job Vancancies')

# Format x-axis to show month/year
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

# Set x-axis limits
ax1.set_xlim([datetime(2009, 10, 1), df_job_vacancy_sa.index.max()])

# Add grid to the main axis
ax1.grid(True)

# Rotate x-axis labels
plt.setp(ax1.get_xticklabels(), rotation=90)

# Add legends
ax1.legend(loc='upper left')

# Adjust layout to ensure the plot is centered and well-fitted on the A4 page
plt.tight_layout(pad=2.0)

# Save the current figure to the PDF
pdf.savefig(fig, bbox_inches='tight')

# Close the plot
plt.close()

In [ ]:
# Figure 2: Labour Force Summary
fig, ax1 = plt.subplots(figsize=(14, 8))

# Add page title
fig.suptitle('Figure 2: Labour Force, Summary', fontsize=16, fontweight='bold', y=0.95)

# Plot total labour force and total employed
ax1.plot(df_labour_sa.index, df_labour_sa['Labour force total'], label='Labour Force Total', color='tab:blue')
ax1.plot(df_labour_sa.index, df_labour_sa['Employed total'], label='Employed Total', color='tab:green')

# Create a second y-axis for total unemployed
ax2 = ax1.twinx()
ax2.plot(df_labour_sa.index, df_labour_sa['Unemployed total'], label='Unemployed Total (RHS)', color='tab:red', linestyle='--')

# Set labels for x and y axes
ax1.set_xlabel('Date')
ax1.set_ylabel('(Thousands)')
ax2.set_ylabel('(Thousands)', color='tab:red')

# Set title for the axes
ax1.set_title('Labour Force')

# Format x-axis to show month/year
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

# Set x-axis limits
ax1.set_xlim([datetime(2009, 10, 1), df_labour_sa.index.max()])

# Add grid to the main axis
ax1.grid(True)

# Rotate x-axis labels
plt.setp(ax1.get_xticklabels(), rotation=90)

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Adjust layout to ensure the plot is centered and well-fitted on the A4 page
plt.tight_layout(pad=2.0)

# Save the current figure to the PDF
pdf.savefig(fig, bbox_inches='tight')

# Close the plot
plt.close()


In [43]:
pdf.close()
